# Face Detector (work in progress)
### Testing face detector using "face_angles.jpg"

In [1]:
# getting image path
# try with flowers.jpg as well (doesn't have any faces)
current = %pwd
from pathlib import Path
path = Path(current)
imagePath = str(path.parent.absolute())
image_name = "face_angles.jpg"
imagePath += '\\' + image_name
imagePath

'c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\face_angles.jpg'

In [2]:
cascPath = "haarcascade_frontalface_default.xml"
cascPath_m = "haarcascade_mouth.xml"
cascPath_e = "haarcascade_eye.xml"

In [3]:
import cv2

In [4]:
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)
mouthCascade = cv2.CascadeClassifier(cascPath_m)
eyeCascade = cv2.CascadeClassifier(cascPath_e)

# Read the image
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

mouths = mouthCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

eyes = eyeCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

In [20]:
print ("Found {0} faces!".format(len(faces)))

# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

print ("Found {0} mouths!".format(len(mouths)))

# Draw a rectangle around the mouths
for (x, y, w, h) in mouths:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

print ("Found {0} eyes!".format(len(eyes)))

# Draw a rectangle around the eyes
for (x, y, w, h) in eyes:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)

Found 9 faces!
Found 22 mouths!
Found 13 eyes!


In [24]:
# input two rectangles
def temp_contains(R1, R2):

    # recalculating the sides from (x1, y1, width, height)
    # to (x1, y1, x2, y2)
    R1X2 = R1[0] + R1[2]
    R1Y2 = R1[1] + R1[3]
    R2X2 = R2[0] + R2[2]
    R2Y2 = R2[1] + R2[3]

    new_x1 = min(R1[0], R2[0])
    new_x2 = max(R1X2, R2X2) - new_x1 # width
    new_y1 = min(R1[1], R2[1])
    new_y2 = max(R1Y2, R2Y2) - new_y1 # height

    New_rect = [new_x1, new_y1, new_x2, new_y2]

    if (R1[0] >= R2X2) or (R1X2 <= R2[0]) or (R1Y2 <= R2[1]) or (R1[1] >= R2Y2):
        # not valid. not contained!
        return False, []
    else:
        # valid. contains
        return True, New_rect


In [25]:
for face in faces:
    for mouth in mouths:
        Valid, mesh_rect = temp_contains(face, mouth)
        if Valid:
            for eye in eyes:
                second_Valid, new_mesh_rect = temp_contains(mesh_rect, eye)
                if second_Valid:
                    x = new_mesh_rect[0]
                    y = new_mesh_rect[1]
                    w = new_mesh_rect[2]
                    h = new_mesh_rect[3]
                    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 2)

                


In [26]:
# runs reasonably well
# notice how it doesn't really work on faces that are turned 90 degrees.

cv2.imshow("Features found", image)
cv2.waitKey(0)


-1

# Face As A Percentage of the Video

In [42]:
# first, get the path of the video in question
# getting video path
current = %pwd
from pathlib import Path
path = Path(current)
filename = str(path.parent.absolute())
# video_name = "American Grilled Cheese Vs. French Grilled Cheese.mp4"
video_name = "Binging with Babish_ Patrick's Briefcase from SpongeBob SquarePants.mp4"
filename += '\\' + video_name
filename

"c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\Binging with Babish_ Patrick's Briefcase from SpongeBob SquarePants.mp4"

In [43]:
import cv2
import numpy as np

In [44]:
# input two rectangles
def contains(R1, R2):

    # recalculating the sides from (x1, y1, width, height)
    # to (x1, y1, x2, y2)
    R1X2 = R1[0] + R1[2]
    R1Y2 = R1[1] + R1[3]
    R2X2 = R2[0] + R2[2]
    R2Y2 = R2[1] + R2[3]

    new_x1 = min(R1[0], R2[0])
    new_x2 = max(R1X2, R2X2) - new_x1 # width
    new_y1 = min(R1[1], R2[1])
    new_y2 = max(R1Y2, R2Y2) - new_y1 # height

    New_rect = [new_x1, new_y1, new_x2, new_y2]

    if (R1[0] >= R2X2) or (R1X2 <= R2[0]) or (R1Y2 <= R2[1]) or (R1[1] >= R2Y2):
        # not valid. not contained!
        return False, []
    else:
        # valid. contains
        return True, New_rect


In [45]:
def face_detector(image):
    
    cascPath = "haarcascade_frontalface_default.xml"
    cascPath_m = "haarcascade_mouth.xml"
    cascPath_e = "haarcascade_eye.xml"

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)
    mouthCascade = cv2.CascadeClassifier(cascPath_m)
    eyeCascade = cv2.CascadeClassifier(cascPath_e)

    # Read the image
    # image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # for (x, y, w, h) in faces:
    #     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Detect mouths in image
    mouths = mouthCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
    )

    # for (x, y, w, h) in mouths:
    #     cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Detect eyes in image
    eyes = eyeCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
    )

    # for (x, y, w, h) in eyes:
    #     cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # yuck. if you can make this faster, go ahead
    # makes sure the face, mouth, and eye rectangles contain each other.
    # This ensures a face
    for face in faces:
        for mouth in mouths:
            Valid, mesh_rect = contains(face, mouth)
            if Valid:
                for eye in eyes:
                    second_Valid, new_mesh_rect = contains(mesh_rect, eye)
                    if second_Valid:
                        return True
                        # x = new_mesh_rect[0]
                        # y = new_mesh_rect[1]
                        # w = new_mesh_rect[2]
                        # h = new_mesh_rect[3]
                        # cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 0), 2)


    # if len(faces) >= 1:
    #     cv2.imshow("Faces found", image)
    #     cv2.waitKey(0)

    # if it detects at least one face, return true
    # it doesn't have to be the host itself
    # if len(faces) >= 1:
    #     return True

    return False


In [46]:
# input: the video file
# output: length of video file in seconds

def find_video_length(video):

    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return frame_count / fps, fps


In [47]:
def face_percentage(filename):

    # person_counter counts number of frames person was in shot
    person_counter = 0
    # no_person_counter counts number of frames where person is not in shot
    no_person_counter = 0

    my_video = cv2.VideoCapture(filename)
    if (my_video.isOpened() == False):
        print("Error opening video stream or file")

    video_length, video_fps = find_video_length(my_video)

    nth_frame = video_fps // 2
    frame_counter = 0

    while (my_video.isOpened()):
        
        ret, frame = my_video.read()
        if frame_counter % nth_frame == 0:

            # Capture frame-by-frame
            if ret == True:
                if face_detector(frame):

                    print(frame_counter / video_fps)
                    person_counter += 1
                else:
                    no_person_counter += 1
            else:
                break

        if ret == False:
            break

        frame_counter += 1

    return (person_counter * 100) / (no_person_counter + person_counter)




In [48]:
filename

"c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\Binging with Babish_ Patrick's Briefcase from SpongeBob SquarePants.mp4"

In [49]:
face_percentage(filename)

117.90945833333332


0.14641288433382138